In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

# Understanding The Data

In [ ]:
# data pre-processing of the family income / comfortability dataset
df = pd.read_excel("fbc_data_2024.xlsx", sheet_name='County', header=1)
ut_df = df[df["State abv."] == 'UT']
cache_county_df = ut_df[ut_df['County'] == 'Cache County']

In [ ]:
def CostOfLivingPerCounty(county_name, plot=False):
    specific_county_df = ut_df[ut_df['County'] == county_name]
    family_types = specific_county_df['Family']
    
    monthly_total_costs = specific_county_df['Total']
    yearly_total_costs = specific_county_df['Total.1']

    print(f"\nCosts for {county_name}:")
    for family, monthly_cost, yearly_cost in zip(family_types, monthly_total_costs, yearly_total_costs):
        print(f"\tFamily Type: {family}, Monthly Cost: ${monthly_cost}, Yearly Cost: ${yearly_cost}")

    if (plot):
        # Monthly costs
        plt.figure(figsize=(12, 8))
        plt.bar(family_types, monthly_total_costs, color='skyblue')
        plt.xlabel('Family Type')
        plt.ylabel('Total Cost of Living ($)')
        plt.title(f'Monthly Cost of Living by Family Type in {county_name}')
        plt.xticks(rotation=45)
        plt.show()

        # Yearly costs
        plt.figure(figsize=(12, 8))
        plt.bar(family_types, yearly_total_costs, color='lightgreen')
        plt.xlabel('Family Type')
        plt.ylabel('Yearly Cost of Living ($)')
        plt.title(f'Yearly Cost of Living by Family Type in {county_name}')
        plt.xticks(rotation=45)
        plt.show()

CostOfLivingPerCounty("Cache County", plot=True)
CostOfLivingPerCounty("Salt Lake County", plot=True)

In [ ]:
def TopCountiesByCost(num, descending=True):
    average_costs = {}
    
    for county in ut_df['County'].unique():
        county_data = ut_df[ut_df['County'] == county]
        average_monthly_cost = county_data['Total'].mean()
        average_costs[county] = average_monthly_cost
        
    top_counties = sorted(average_costs, key=average_costs.get, reverse=descending)[:num]

    adj = 'most' if descending else 'least'
    print(f"\nTop {num} {adj} expensive counties by monthly cost:")
    for county in top_counties:
        print(f"\t{county}: ${average_costs[county]:.2f}")

TopCountiesByCost(11)
TopCountiesByCost(11, descending=False)

In [ ]:
# pre-processing wages dataset
df = pd.read_excel("wages.xlsx")
wage_df = df[~df['Area Name'].isin(['United States', 'Statewide'])]

def wagesAnalysis(wage_data, inexperienced=True):
    adj = 'Inexperienced' if inexperienced else 'Median'
    
    # analysis of average salary by job title
    average_salary_by_job = wage_data.groupby('Job Title')[f'Annual {adj}'].mean().reset_index()
    best_paying_jobs = average_salary_by_job.sort_values(by=f'Annual {adj}', ascending=False).head()
    worst_paying_jobs = average_salary_by_job.sort_values(by=f'Annual {adj}', ascending=True).head()
    
    # plotting best and worst paying jobs
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 5))
    sns.barplot(x='Job Title', y=f'Annual {adj}', data=best_paying_jobs, ax=axes[0], color='skyblue')
    axes[0].set_title(f'Best Paying Jobs ({adj})')
    sns.barplot(x='Job Title', y=f'Annual {adj}', data=worst_paying_jobs, ax=axes[1], color='lightgreen')
    axes[1].set_title(f'Worst Paying Jobs ({adj})')
    for ax in axes:
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    plt.tight_layout()

    # analysis of average salary by area
    average_salary_by_area = wage_data.groupby('Area Name')[f'Annual {adj}'].mean().reset_index()
    best_locations = average_salary_by_area.sort_values(by=f'Annual {adj}', ascending=False).head()
    worst_locations = average_salary_by_area.sort_values(by=f'Annual {adj}', ascending=True).head()
    
    # plotting best and worst locations for salaries vertically
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 5))
    sns.barplot(x='Area Name', y=f'Annual {adj}', data=best_locations, ax=axes[0], color='skyblue')
    axes[0].set_title(f'Best Locations by Average Salary ({adj})')
    sns.barplot(x='Area Name', y=f'Annual {adj}', data=worst_locations, ax=axes[1], color='lightgreen')
    axes[1].set_title(f'Worst Locations by Average Salary ({adj})')
    for ax in axes:
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

wagesAnalysis(wage_df, inexperienced=True)
wagesAnalysis(wage_df, inexperienced=False)

In [ ]:
# rename "Area Name" in wage_df to "County" to match ut_df
wage_df.rename(columns={'Area Name': 'County'}, inplace=True)

# merge the datasets based on the county
combined_df = pd.merge(wage_df, ut_df, on='County')

# ['Cache', 'Central Southwest Utah', 'Eastern Utah', 'Ogden-Clearfield Metro', 'Provo-Orem Metro', 'Salt Lake Metro', 'St George Metro']
# ['Beaver County', 'Box Elder County', 'Cache County', 'Carbon County', 'Daggett County', 'Davis County', 'Duchesne County', 'Emery County', 'Garfield County', 'Grand County', 'Iron County', 'Juab County', 'Kane County', 'Millard County', 'Morgan County', 'Piute County', 'Rich County', 'Salt Lake County', 'San Juan County', 'Sanpete County', 'Sevier County', 'Summit County', 'Tooele County', 'Uintah County', 'Utah County', 'Wasatch County', 'Washington County', 'Wayne County', 'Weber County']